In [1]:
import json
import pandas as pd
import re
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

import os
from IPython.display import display, Image

In [7]:
year =  [str(i) for i in range(1980, 2016) ]

df = pd.DataFrame()

for y in tqdm(year):
    file_path = 'movie_data_per_year/'+y+'.txt'  # Replace with the path to your text file


        
    input_file = file_path  # Replace with the path to your input file
    output_file = file_path  # Replace with the path to your output file

        
    
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

        
    
    jump = 0
    
    while not("}" in lines[jump]):
        jump += 1
        
    jump += 1
        
        
    for i in range(0, len(lines), jump):
        test_text = lines[i: i+jump]
        fin_text  = ''.join(test_text)
        

        cleaned_text = re.sub(r'ObjectId\("([^"]+)"\)', r'"\1"', fin_text)

        # Parse the text as a JSON object
        json_obj = json.loads(cleaned_text)
        # Convert the JSON object to a pandas DataFrame
        json_obj["imdbID"] += ".jpg"
        df = df.append(json_obj, ignore_index = True)

df = df.drop("Poster", axis = 1)
df = df.rename(columns={'imdbID': 'Poster'})

100%|██████████████████████████████████████████████████████████████████████████████████| 36/36 [00:30<00:00,  1.20it/s]


In [8]:
df.to_csv("complete_data_movie.csv", encoding='utf-8', index=False)

In [128]:
df_movie = pd.read_csv("complete_data_movie.csv")

In [129]:
#only keep rows with movie type (remove series and games)
df_movie = df_movie[df_movie["Type"] == "movie"]


# changes actors and directors to lists and remove useless columns (id, Rated, imdbVotes, Response, type, Metascore)
df_movie['Actors'] = df_movie['Actors'].map(lambda x: x.split(', ') if isinstance(x, str) else np.nan)
df_movie['Director'] = df_movie['Director'].fillna('').apply(lambda x: [director.strip() for director in x.split(',')])
df_movie = df_movie[['Title','Genre', 'Actors','Director','Writer','Language','Box_office','Country','Released','Runtime','imdbRating','Awards', 'Poster']]

#drop cols with no actors / genre / poster
df_movie = df_movie.dropna(subset=['Actors','Genre','Poster', 'Released', 'imdbRating', 'Box_office'])

df_movie[0:2]

,Title,Genre,Actors,Director,Writer,Language,Box_office,Country,Released,Runtime,imdbRating,Awards,Poster
0,Stir Crazy,"Comedy, Crime","[Gene Wilder, Richard Pryor, Georg Stanford Br...",[Sidney Poitier],Bruce Jay Friedman,English,"101,500,000",USA,12 Dec 1980,111 min,6.8,1 nomination.,tt0081562.jpg
1,Airplane!,Comedy,"[Kareem Abdul-Jabbar, Lloyd Bridges, Peter Gra...","[Jim Abrahams, David Zucker, Jerry Zucker]","Jim Abrahams (written for the screen by), Davi...",English,"83,400,000",USA,02 Jul 1980,88 min,7.8,Nominated for 1 Golden Globe. Another 2 wins &...,tt0080339.jpg


In [130]:
actor_df = pd.read_csv("actorfilms.csv") #to update the actors list 


In [131]:
def get_actors_by_film(df, film, year):
    return list(df[(df["Film"] == film) & (df["Year"] == year)]["Actor"].values)


same_title = []

for title in tqdm(df_movie["Title"].unique()):
    bulk_year = int(df_movie[df_movie['Title'] == title]["Released"].values[0][-4:])
    
    if title in actor_df[actor_df["Film"] == title]["Film"].unique():
        
        for upt_year in actor_df[actor_df["Film"] == title]["Year"].unique():
        
            if abs(bulk_year - upt_year) < 2:
                title_index = df_movie[(df_movie["Title"] == title) & (df_movie["Released"].str.contains(str(bulk_year)))].index[0]
                df_movie.at[title_index, 'Actors'] = get_actors_by_film(actor_df, title, bulk_year)

            else:
                    
                new_row = {"Title": title, 
                           "Year_bulk": [bulk_year],
                           "Year_upt": [upt_year]
                          }
                same_title.append(new_row)


100%|██████████████████████████████████████████████████████████████████████████████| 6012/6012 [05:14<00:00, 19.09it/s]


In [132]:
print(len(same_title))
for el in same_title:
    print(el)

888
{'Title': 'Dressed to Kill', 'Year_bulk': [1980], 'Year_upt': [1946]}
{'Title': 'The Jazz Singer', 'Year_bulk': [1980], 'Year_upt': [1927]}
{'Title': 'The Jazz Singer', 'Year_bulk': [1980], 'Year_upt': [1952]}
{'Title': 'The Fog', 'Year_bulk': [1980], 'Year_upt': [2005]}
{'Title': 'Arthur', 'Year_bulk': [1981], 'Year_upt': [2011]}
{'Title': 'Halloween II', 'Year_bulk': [1981], 'Year_upt': [2009]}
{'Title': 'Under the Rainbow', 'Year_bulk': [1981], 'Year_upt': [2013]}
{'Title': 'Pennies from Heaven', 'Year_bulk': [1982], 'Year_upt': [1936]}
{'Title': 'Poltergeist', 'Year_bulk': [1982], 'Year_upt': [2015]}
{'Title': 'Annie', 'Year_bulk': [1982], 'Year_upt': [2014]}
{'Title': 'The Verdict', 'Year_bulk': [1982], 'Year_upt': [1974]}
{'Title': 'The Verdict', 'Year_bulk': [1982], 'Year_upt': [1946]}
{'Title': 'Conan the Barbarian', 'Year_bulk': [1982], 'Year_upt': [2011]}
{'Title': 'Night Shift', 'Year_bulk': [1982], 'Year_upt': [2020]}
{'Title': 'Diner', 'Year_bulk': [1982], 'Year_upt': 

In [133]:
df_movie[df_movie["Title"] == "Ocean's Eleven"]

,Title,Genre,Actors,Director,Writer,Language,Box_office,Country,Released,Runtime,imdbRating,Awards,Poster
3753,Ocean's Eleven,"Crime, Thriller","[Brad Pitt, George Clooney, Julia Roberts, Don...",[Steven Soderbergh],"George Clayton Johnson, Jack Golden Russell, H...","English, Italian, Cantonese","183,405,771",USA,07 Dec 2001,116 min,7.8,4 wins & 20 nominations.,tt0240772.jpg


In [122]:
def get_actors_by_film(df, film, year):
    return list(df[(df["Film"] == film) & (df["Year"] == year)]["Actor"].values)

t = get_actors_by_film(actor_df, "Ocean's Eleven", 2001)
t

['Brad Pitt',
 'George Clooney',
 'Julia Roberts',
 'Don Cheadle',
 'Matt Damon',
 'Andy Garcia',
 'Casey Affleck',
 'Holly Marie Combs',
 'Angie Dickinson',
 'Elliott Gould',
 'Steven Soderbergh',
 'J.P. Manoux',
 'Scott Caan',
 'Bernie Mac',
 'Carl Reiner',
 'Topher Grace',
 'David Leitch',
 'Robin Sachs',
 'Henry Silva']

In [109]:
df_movie_test = df_movie_test[df_movie_test["Type"] == "movie"]


# changes actors and directors to lists and remove useless columns (id, Rated, imdbVotes, Response, type, Metascore)
df_movie_test['Actors'] = df_movie_test['Actors'].map(lambda x: x.split(', ') if isinstance(x, str) else np.nan)
df_movie_test['Director'] = df_movie_test['Director'].fillna('').apply(lambda x: [director.strip() for director in x.split(',')])
df_movie_test = df_movie_test[['Title','Genre', 'Actors','Director','Writer','Language','Box_office','Country','Released','Runtime','imdbRating','Awards', 'Poster']]

df_movie_test[:2]

,Title,Genre,Actors,Director,Writer,Language,Box_office,Country,Released,Runtime,imdbRating,Awards,Poster
0,Stir Crazy,"Comedy, Crime","[Gene Wilder, Richard Pryor, Georg Stanford Br...",[Sidney Poitier],Bruce Jay Friedman,English,"101,500,000",USA,12 Dec 1980,111 min,6.8,1 nomination.,tt0081562.jpg
1,Airplane!,Comedy,"[Kareem Abdul-Jabbar, Lloyd Bridges, Peter Gra...","[Jim Abrahams, David Zucker, Jerry Zucker]","Jim Abrahams (written for the screen by), Davi...",English,"83,400,000",USA,02 Jul 1980,88 min,7.8,Nominated for 1 Golden Globe. Another 2 wins &...,tt0080339.jpg


In [134]:
df_movie.to_csv("complete_data_movie_with_correct_actor2.csv", encoding='utf-8', index=False)


In [139]:
df_movie[df_movie["Title"] == "Ocean's Eleven"]["Actors"]


3753    [Brad Pitt, George Clooney, Julia Roberts, Don...
Name: Actors, dtype: object

,Title,Genre,Actors,Director,Writer,Language,Box_office,Country,Released,Runtime,imdbRating,Awards,Poster
0,Stir Crazy,"Comedy, Crime","[Gene Wilder, Richard Pryor, JoBeth Williams, ...",[Sidney Poitier],Bruce Jay Friedman,English,"101,500,000",USA,12 Dec 1980,111 min,6.8,1 nomination.,tt0081562.jpg
1,Airplane!,Comedy,"[Leslie Nielsen, Lloyd Bridges, Robert Hays, J...","[Jim Abrahams, David Zucker, Jerry Zucker]","Jim Abrahams (written for the screen by), Davi...",English,"83,400,000",USA,02 Jul 1980,88 min,7.8,Nominated for 1 Golden Globe. Another 2 wins &...,tt0080339.jpg
3,Private Benjamin,"Comedy, War","[Goldie Hawn, Sally Kirkland, Armand Assante, ...",[Howard Zieff],"Nancy Meyers, Charles Shyer, Harvey Miller",English,"69,800,000",USA,10 Oct 1980,109 min,6.1,Nominated for 3 Oscars. Another 1 win & 3 nomi...,tt0081375.jpg
4,Coal Miner's Daughter,"Biography, Drama, Music","[Tommy Lee Jones, Beverly D'Angelo, Sissy Spac...",[Michael Apted],"Thomas Rickman (screenplay), Loretta Lynn (aut...",English,"79,900,000",USA,07 Mar 1980,124 min,7.5,Won 1 Oscar. Another 8 wins & 13 nominations.,tt0080549.jpg
5,Smokey and the Bandit II,"Action, Comedy","[Sally Field, Burt Reynolds, Dom DeLuise, Jack...",[Hal Needham],"Hal Needham (characters created by), Robert L....","English, Spanish","66,100,000",USA,15 Aug 1980,100 min,5.1,1 win & 4 nominations.,tt0081529.jpg
